# Introduction

One of the widely used natural language processing task in different business problems is “Text Classification”. The goal of text classification is to automatically classify the text documents into one or more defined categories. Some examples of text classification are:

Understanding audience sentiment from social media,
Detection of spam and non-spam emails,
Auto tagging of customer queries, and
Categorization of news articles into defined topics.

Text Classification is an example of supervised machine learning task since a labelled dataset containing text documents and their labels is used for train a classifier. An end-to-end text classification pipeline is composed of three main components:

## 1. Dataset Preparation:
The first step is the Dataset Preparation step which includes the process of loading a dataset and performing basic pre-processing. The dataset is then splitted into train and validation sets.

## 2. Feature Engineering: 
The next step is the Feature Engineering in which the raw dataset is transformed into flat features which can be used in a machine learning model. This step also includes the process of creating new features from the existing data.

## 3. Model Training:
The final step is the Model Building step in which a machine learning model is trained on a labelled dataset.

## 4. Improve Performance of Text Classifier: 

We will also look at the different ways to improve the performance of text classifiers.

# Readme

Getting your machine ready

Lets implement basic components in a step by step manner in order to create a text classification framework in python. To start with, import all the required libraries.

You would need requisite libraries to run this code – you can install them at their individual official links

Pandas (https://pandas.pydata.org/pandas-docs/stable/install.html)
Scikit-learn (http://scikit-learn.org/stable/install.html)
XGBoost (https://xgboost.readthedocs.io/en/latest/build.html)
TextBlob (https://textblob.readthedocs.io/en/dev/install.html)
Keras(https://keras.io/#installation) 


Load the libraries

In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import  xgboost, textblob, string
import pandas as pd
import numpy as np
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

import re

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## 1. dataset preparation

In [2]:
#load the dataset

trainDf = pd.read_csv("./Dataset/train.csv",encoding = "utf-8")

In [3]:
trainDf.head()

,ID,reviewText,overall
0,1,These are so wonderful to have in the car in t...,5
1,2,XL fits perfect on me over armored riding jack...,5
2,3,Since I had just noticed my wiper blades neede...,3
3,4,I am very satisfied with G110v2. It is comfor...,5
4,5,This amp did a good job for its rating. It was...,3


In [4]:
trainDf.isna().sum()

ID             0
reviewText    26
overall        0
dtype: int64

In [5]:
cleanup_re = re.compile('[^a-zA-Z]+')

def cleanup(sentence):
    sentence = str(sentence)
    sentence = re.sub(r'\s+',' ',sentence) #\s is for white spaces
    sentence = re.sub('[\d]','',sentence)  #\d is for digits
    #sentence = sentence.lower()
    sentence = cleanup_re.sub(' ', sentence).strip()
    #sentence = " ".join(nltk.word_tokenize(sentence))
    return sentence

trainDf["text"] = trainDf["reviewText"].apply(cleanup)

In [6]:
trainDf.drop(['reviewText'], axis=1, inplace=True)

Next, we will split the dataset into training and validation sets so that we can train and test classifier. Also, we will encode our target column so that it can be used in machine learning models.



In [7]:
#split the dataset intro training an dvalidation datasets

train_X, valid_X, train_y, valid_y = model_selection.train_test_split(trainDf['text'], trainDf['overall'])


#label encode the target variable
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

# 2. Feature Engineering

The next step is the feature engineering step. In this step, raw text data will be transformed into feature vectors and new features will be created using the existing dataset. We will implement the following different ideas in order to obtain relevant features from our dataset.

2.1 Count Vectors as features

2.2 TF-IDF Vectors as features

Word level

N-Gram level

Character level

2.3 Word Embeddings as features

2.4 Text / NLP based features

2.5 Topic Models as features

Lets look at the implementation of these ideas in detail.

## 2.1 Count Vectors as features

Count Vector is a matrix notation of the dataset in which every row represents a document from the corpus, every column represents a term from the corpus, and every cell represents the frequency count of a particular term in a particular document.



In [8]:
# Create a count vectorizer object

count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDf['text'])

#transform the training and validation data using count vectorizer object

xtrain_count = count_vect.transform(train_X)
xvalid_count = count_vect.transform(valid_X)

## 2.2 TF-IDF Vectors as features

TF-IDF score represents the relative importance of a term in the document and the entire corpus. TF-IDF score is composed by two terms: the first computes the normalized Term Frequency (TF), the second term is the Inverse Document Frequency (IDF), computed as the logarithm of the number of the documents in the corpus divided by the number of documents where the specific term appears.

TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document)
IDF(t) = log_e(Total number of documents / Number of documents with term t in it)

TF-IDF Vectors can be generated at different levels of input tokens (words, characters, n-grams)

### a. Word Level TF-IDF : 
Matrix representing tf-idf scores of every term in different documents

### b. N-gram Level TF-IDF :
N-grams are the combination of N terms together. This Matrix representing tf-idf scores of N-grams

### c. Character Level TF-IDF :
Matrix representing tf-idf scores of character level n-grams in the corpus

In [9]:
# Word level tf-idf

tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDf['text'])
xtrain_tfidf = tfidf_vect.transform(train_X)
xvalid_tfidf = tfidf_vect.transform(valid_X)


#ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDf['text'])
xtrain_tfidf_ngram = tfidf_vect_ngram.transform(train_X)
xvalid_tfidf_ngram = tfidf_vect_ngram.transform(valid_X)


# Characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDf['text'])
xtrain_tfidf_ngram_chars = tfidf_vect_ngram_chars.transform(train_X)
xvalid_tfidf_ngram_chars = tfidf_vect_ngram_chars.transform(valid_X)

In [10]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('/Users/ankitjain/Desktop/INSOFE/Extra/Cute/Cute (24.10.2018)/CSE7321c_Deep_Learning and Text_Mining_CUTe/glove/glove.6B.300d.txt.word2vec')):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(trainDf['text'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_X), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_X), maxlen=70)

# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

## 2.4 Text / NLP based features

A number of extra text based features can also be created which sometimes are helpful for improving text classification models. Some examples are:

Word Count of the documents – total number of words in the documents
Character Count of the documents – total number of characters in the documents
Average Word Density of the documents – average length of the words used in the documents
Puncutation Count in the Complete Essay – total number of punctuation marks in the documents
Upper Case Count in the Complete Essay – total number of upper count words in the documents
Title Word Count in the Complete Essay – total number of proper case (title) words in the documents
Frequency distribution of Part of Speech Tags:
Noun Count
Verb Count
Adjective Count
Adverb Count
Pronoun Count
These features are highly experimental ones and should be used according to the problem statement only.

In [ ]:
trainDf['char_count'] = trainDf['text'].apply(len)

In [ ]:
trainDf['word_count'] = trainDf['text'].apply(lambda x: len(x.split()))
trainDf['word_density'] = trainDf['char_count']/(trainDf['word_count']+1)

In [ ]:
trainDf['punctuation_count'] = trainDf['text'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation)))

In [ ]:
trainDf['tite_word_count'] = trainDf['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))

In [ ]:
trainDf['upper_case_word_count'] = trainDf['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [ ]:
pos_family = {
    'noun' : ['NN', 'NNS', 'NNP', 'NNPS'],
    'pron' : ['PRP', 'PRP$', 'WP', 'WP$'],
    'verb' : ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'],
    'adj' : ['JJ', 'JJR', 'JJS'],
    'adv' : ['RB', 'RBR', 'RBS', 'WRB']
}

#function to check and get the part of the speech tag count of a word in a given sentence

def check_pos_tag(x, flag):
    cnt=0
    try:
        wiki = textblob.TextBlob(x)
        for tup in wiki.tags:
            ppo=list(tup)[1]
            if ppo in pos_family[flag]:
                cnt+=1
    except:
        pass
    return cnt



trainDf['noun_count'] = trainDf['text'].apply(lambda x: check_pos_tag(x, 'noun'))
trainDf['verb_count'] = trainDf['text'].apply(lambda x: check_pos_tag(x, 'verb'))
trainDf['adj_count'] = trainDf['text'].apply(lambda x : check_pos_tag(x, 'adj'))
trainDf['adv_count'] = trainDf['text'].apply(lambda x : check_pos_tag(x, 'adv'))
trainDf['pron_count'] = trainDf['text'].apply(lambda x: check_pos_tag(x, 'pron'))

In [ ]:
trainDf

### 2.5 Topic Models as features

Topic Modelling is a technique to identify the groups of words (called a topic) from a collection of documents that contains best information in the collection. I have used Latent Dirichlet Allocation for generating Topic Modelling Features. LDA is an iterative model which starts from a fixed number of topics. Each topic is represented as a distribution over words, and each document is then represented as a distribution over topics. Although the tokens themselves are meaningless, the probability distributions over words provided by the topics provide a sense of the different ideas contained in the documents. One can read more about topic modelling here

Lets see its implementation:

In [11]:
# train a LDA model

lda_model = decomposition.LatentDirichletAllocation(n_components=20 , learning_method='online', max_iter=20)
x_topics = lda_model.fit_transform(xtrain_count)
topic_word = lda_model.components_
vocab = count_vect.get_feature_names()


# view the topic models
n_top_words =10
topic_summary = []
for i, topic_dict in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dict)][:-(n_top_words+1):-1]
    topic_summary.append(' '.join(topic_words))

KeyboardInterrupt: 

# 3. Model Building

The final step in the text classification framework is to train a classifier using the features created in the previous step. There are many different choices of machine learning models which can be used to train a final model. We will implement following different classifiers for this purpose:

Naive Bayes Classifier
Linear Classifier
Support Vector Machine
Bagging Models
Boosting Models
Shallow Neural Networks
Deep Neural Networks
Convolutional Neural Network (CNN)
Long Short Term Modelr (LSTM)
Gated Recurrent Unit (GRU)
Bidirectional RNN
Recurrent Convolutional Neural Network (RCNN)
Other Variants of Deep Neural Networks
Lets implement these models and understand their details. The following function is a utility function which can be used to train a model. It accepts the classifier, feature_vector of training data, labels of training data and feature vectors of valid data as inputs. Using these inputs, the model is trained and accuracy score is computed.

In [12]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net = False):
    
    #fit the training dataset on the  classifier
    classifier.fit(feature_vector_train, label)
    
    #predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
        
    return metrics.accuracy_score(predictions, valid_y)

## 3.1 Naive Bayes

Implementing a naive bayes model using sklearn implementation with different features

Naive Bayes is a classification technique based on Bayes’ Theorem with an assumption of independence among predictors. A Naive Bayes classifier assumes that the presence of a particular feature in a class is unrelated to the presence of any other feature here .

In [13]:
# Naive bayes on count vectors

accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)

print("NB, count vectors:", accuracy)


#Naive bayes on word level tf idf vectors


accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)

print("NB, wordlevel tf idf:", accuracy)


# Naive bayes on ngram level tf idf vectors

accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)

print("NB, n gram vectors:", accuracy)


# Naive bayes on character level tf idf vectors


accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)

print("NB, char level vectors:",accuracy)

NB, count vectors: 0.6144584565883024
NB, wordlevel tf idf: 0.6116041309875967
NB, n gram vectors: 0.626187139965748
NB, char level vectors: 0.6007576937049146


## 3.2 Linear Classifier

Implementing a Linear Classifier (Logistic Regression)

Logistic regression measures the relationship between the categorical dependent variable and one or more independent variables by estimating probabilities using a logistic/sigmoid function. One can read more about logistic regression here

In [14]:
# Linaer classifier on count vectors

accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)

print("LR, count vectors:", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("LR, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("LR, CharLevel Vectors: ", accuracy)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LR, count vectors: 0.632622346774612
LR, WordLevel TF-IDF:  0.6515646893974778
LR, N-Gram Vectors:  0.6423270538170118
LR, CharLevel Vectors:  0.6414967045513519


## 3.3 Implementing a SVM Model

Support Vector Machine (SVM) is a supervised machine learning algorithm which can be used for both classification or regression challenges. The model extracts a best possible hyper-plane / line that segregates the two classes. One can read more about it here

In [ ]:
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("SVM, N-Gram Vectors: ", accuracy)

## 3.4 Bagging Model

Implementing a Random Forest Model

Random Forest models are a type of ensemble models, particularly bagging models. They are part of the tree based model family. One can read more about Bagging and random forests here

In [15]:
# RF on count vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)

print("RF, count vectors:", accuracy)


# RF on word level tfidf vectors

accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)

print("RF, word vlevel tfidf :", accuracy)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


KeyboardInterrupt: 

## 3.5 Boosting Model

Implementing Xtereme Gradient Boosting Model

Boosting models are another type of ensemble models part of tree based models. Boosting is a machine learning ensemble meta-algorithm for primarily reducing bias, and also variance in supervised learning, and a family of machine learning algorithms that convert weak learners to strong ones. A weak learner is defined to be a classifier that is only slightly correlated with the true classification (it can label examples better than random guessing). Read more about these models here

In [ ]:
# Extereme Gradient Boosting on Count Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_count.tocsc(), train_y, xvalid_count.tocsc())
print("Xgb, Count Vectors: ", accuracy)

# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print("Xgb, WordLevel TF-IDF: ", accuracy)

# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print("Xgb, CharLevel Vectors: ", accuracy)

## 3.6 Shallow Neural Networks

A neural network is a mathematical model that is designed to behave similar to biological neurons and nervous system. These models are used to recognize complex patterns and relationships that exists within a labelled data. A shallow neural network contains mainly three types of layers – input layer, hidden layer, and output layer. Read more about neural networks here

In [16]:
def create_model_architecture(input_size):
    
    #create input layer
    input_layer = layers.Input((input_size,), sparse=True)
    
    
    #create hidden layer
    
    hidden_layer = layers.Dense(100, activation='relu')(input_layer)
    
    
    
    #create output layer
    
    output_layer = layers.Dense(1, activation='sigmoid')(hidden_layer)
    
    
    classifier = models.Model(input_layer, output_layer)
    classifier.compile(optimizer= optimizers.Adam(), loss='binary_crossentropy')
    return classifier


classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])
accuracy = train_model(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, is_neural_net=True)

print("NN, ngram level tf idf vectors:", accuracy)

Epoch 1/1
57806/57806 [==============================] - 8s 141us/step - loss: -33.9574
NN, ngram level tf idf vectors: 0.03855934402408013


## 3.7 Deep Neural Networks

Deep Neural Networks are more complex neural networks in which the hidden layers performs much more complex operations than simple sigmoid or relu activations. Different types of deep learning models can be applied in text classification problems.

### 3.7.1 Convolutional Neural Network

In Convolutional neural networks, convolutions over the input layer are used to compute the output. This results in local connections, where each region of the input is connected to a neuron in the output. Each layer applies different filters and combines their results.

In [17]:
def create_cnn():
    
    #Add an input layer
    input_layer = layers.Input((70,))
    
    
    #Add the owrd embedding layer
    
    embedding_layer = layers.Embedding(len(word_index)+1, 300, weights=[embedding_matrix], trainable= False)(input_layer)
    
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    
    #Add the convolutional layer
    
    conv_layer = layers.Convolution1D(100, 3, activation='relu')(embedding_layer)
    
    
    #Add the pooling layer
    
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)
    
    
    
    #Add the output layers
    
    output_layer1 = layers.Dense(50, activation='relu')(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation='sigmoid')(output_layer1)
    
    
    # compile the mdoel
    model = models.Model(input_layer, output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    
    return model

classifier = create_cnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("CNN, Word Embeddings",  accuracy)

Epoch 1/1
57806/57806 [==============================] - 29s 498us/step - loss: -36.5082
CNN, Word Embeddings 0.03855934402408013


### 3.7.2 Recurrent Neural Network – LSTM

Unlike Feed-forward neural networks in which activation outputs are propagated only in one direction, the activation outputs from neurons propagate in both directions (from inputs to outputs and from outputs to inputs) in Recurrent Neural Networks. This creates loops in the neural network architecture which acts as a ‘memory state’ of the neurons. This state allows the neurons an ability to remember what have been learned so far.

The memory state in RNNs gives an advantage over traditional neural networks but a problem called Vanishing Gradient is associated with them. In this problem, while learning with a large number of layers, it becomes really hard for the network to learn and tune the parameters of the earlier layers. To address this problem, A new type of RNNs called LSTMs (Long Short Term Memory) Models have been developed.

In [18]:
def create_rnn_lstm():
    input_layer = layers.Input((70,))

    ##Add the owrd embedding layer
    
    embedding_layer = layers.Embedding(len(word_index)+1, 300, weights=[embedding_matrix], trainable = False)(input_layer)
    
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    
    #Add the lstm layer
    
    lstm_layer = layers.LSTM(100)(embedding_layer)
    
    
    #Add the output layer
    output_layer1= layers.Dense(50, activation='relu')(lstm_layer)
    output_layer1= layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation='sigmoid')(output_layer1)
    
    
    
    #Compile the model
    
    model= models.Model(input_layer, output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_lstm()
accuracy= train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)

print("RNN lstm, word embeddings :", accuracy)

Epoch 1/1
57806/57806 [==============================] - 138s 2ms/step - loss: -36.5779
RNN lstm, word embeddings : 0.03855934402408013


### 3.7.3 Recurrent Neural Network – GRU

Gated Recurrent Units are another form of recurrent neural networks. Lets add a layer of GRU instead of LSTM in our network.

In [19]:
def create_rnn_gru():
    # Add an input layer
    input_layer = layers.Input((70,))
    
    #Add the word embedding layer
    embedding_layer = layers.Embedding(len(word_index)+1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    
    #Add the gru layer
    
    lstm_layer = layers.GRU(100)(embedding_layer)
    
    
    #Add the output layers
    output_layer1= layers.Dense(50, activation='relu')(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation='sigmoid')(output_layer1)
    
    
    #compile thw model
    model = models.Model(input_layer, output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    
    return model
classifier = create_rnn_gru()
accuracy= train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("RNN GRU , word embedding :", accuracy)

Epoch 1/1
57806/57806 [==============================] - 115s 2ms/step - loss: -36.4685
RNN GRU , word embedding : 0.03855934402408013


### 3.7.4 Bidirectional RNN

RNN layers can be wrapped in Bidirectional layers as well. Lets wrap our GRU layer in bidirectional layer.

In [20]:
def create_bidirectional_rnn():
    #add the input layer
    input_layer = layers.Input((70,))
    
    #Add the word embedding layer
    embedding_layer = layers.Embedding(len(word_index)+1,300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    #Add the lstm layer
    
    lstm_layer = layers.Bidirectional(layers.GRU(100))(embedding_layer)
    
    
    #add the output layers
    output_layer1 = layers.Dense(50, activation='relu')(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation='sigmoid')(output_layer1)
    
    #compile the model
    model = models.Model(input_layer, output_layer2)
    model.compile(optimizer= optimizers.Adam(), loss= 'binary_crossentropy')
    
    return model
classifier = create_bidirectional_rnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("RNN bidirectional, word embeddings:", accuracy)

Epoch 1/1
57806/57806 [==============================] - 144s 2ms/step - loss: -36.5597
RNN bidirectional, word embeddings: 0.03855934402408013


### 3.7.5 Recurrent Convolutional Neural Network

Once the essential architectures have been tried out, one can try different variants of these layers such as recurrent convolutional neural network. Another variants can be:

Hierarichial Attention Networks
Sequence to Sequence Models with Attention
Bidirectional Recurrent Convolutional Neural Networks
CNNs and RNNs with more number of layers

In [21]:
def create_rcnn():
    #add the input layer 
    input_layer = layers.Input((70,))
    
    #add the word embedding layer
    embedding_layer = layers.Embedding(len(word_index)+1, 300, weights=[embedding_matrix], trainable = False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    #Add the recurrent layer 
    rnn_layer = layers.Bidirectional(layers.GRU(50, return_sequences=True))(embedding_layer)
    
    #Add the convolutional layer
    conv_layer= layers.Convolution1D(100, 3, activation='relu')(embedding_layer)
    
    
    #add the pooling layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)
    
    
    #Add the output layers
    output_layer1 = layers.Dense(50, activation='relu')(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation='sigmoid')(output_layer1)
    
    #compile the model
    
    model = models.Model(input_layer, output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    return model
classifier = create_rcnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("RCNN, word embeddings:",accuracy)

Epoch 1/1
57806/57806 [==============================] - 29s 506us/step - loss: -36.6543
RCNN, word embeddings: 0.03855934402408013
